In [ ]:
from kafka import KafkaConsumer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import json
from IPython.display import clear_output

In [ ]:
import nltk 
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
STOPWORDS = set(stopwords.words('portuguese'))
STOPWORDS2 = ['RT', 'https', 'co']

In [ ]:
def remove_stopwords(texto):
    texto_limpo = ' '.join(palavra for palavra in texto.split() if palavra not in STOPWORDS)
    texto_final = ' '.join(palavra for palavra in texto_limpo.split() if palavra not in STOPWORDS2)
    return texto_final

In [ ]:
#configuração do kafka
brokers = ['localhost:9092']
topico = 'dados-tweets'

consumer = KafkaConsumer(topico, group_id = 'group1', bootstrap_servers = brokers)

Nuvem de Palavras

In [ ]:
#geração da nuvem de palavras em tempo real
frases = ''
for messagem in consumer:
    texto = json.loads(messagem.value.decode('utf-8'))
    frases = frases + remove_stopwords(texto['tweet'])
    clear_output()
    wordcloud = WordCloud(max_font_size=100, width = 1520, height = 535).generate(frases)
    plt.figure(figsize=(16,9))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

PySpark

In [ ]:
from pyspark.sql import SparkSession, SQLContext, Row
import pyspark.sql.functions as f
from operator import add

In [ ]:
def init_spark():
    spark = SparkSession.builder.appName("Tweets").getOrCreate()
    sc = spark.sparkContext
    sql = SQLContext(sc)
    return spark, sc, sql

In [ ]:
spark, sc, sql = init_spark()

In [ ]:
frases = frases.lower()
lista_palavras = frases.split()

In [ ]:
rdd1 = sc.parallelize(lista_palavras)
row_rdd = rdd1.map(lambda x: Row(x))
df = sql.createDataFrame(row_rdd,['palavras'])
df.show()

In [ ]:
df.withColumn('word', f.explode(f.split(f.col('palavras'), ' '))).groupBy('word').count().sort('count', ascending=False).show()


In [ ]:
palavras_contadas = sc.parallelize(lista_palavras).map(lambda word: (word, 1)).reduceByKey(add)


In [ ]:
palavras_contadas.saveAsTextFile('teste')

In [ ]:
palavras_contadas.collect()